In [1]:
from py2neo import Node, Relationship, Graph, NodeMatcher, RelationshipMatcher
from jshen.io import load_yaml

pwd = load_yaml("../.private/pwd.yaml")["pwd"]
graph = Graph(
    password=pwd
)

In [2]:
from jie import *

In [3]:
ent1 = EntityLimit("大尺度生态空间", "任务")
ent2 = EntityLimit("北运河湿地公园", "Object")
rel = TripleLimit(ent1, ent2, "包含")

In [4]:
# Triple(Entity(大尺度生态空间, 任务) - 包含 - Entity(北运河湿地公园, Object))
print(rel)

Triple(Entity(大尺度生态空间, 任务) - 包含 - Entity(北运河湿地公园, Object))


In [5]:
node_matcher = NodeMatcher(graph)

In [25]:
relation_matcher = RelationshipMatcher(graph) # 生成关系匹配器

## 向neo4j添加数据

查询某实体是否存在, 不存在则创建

In [19]:
from typing import Union
def find_entity(ent: EntityLimit) -> Union[None, EntityLimit]:
    label = ent.type
    d = deepcopy(ent.__dict__)
    d.pop("type")
    return NodeMatcher(graph).match(label, **d).first()

In [9]:
print(find_entity(rel.subject))

None


In [16]:
def create_node(graph, label, **kwargs):
    node = Node(label, **kwargs)
    graph.create(node)
    return node

In [11]:
from copy import deepcopy

In [20]:
def get_node(ent: EntityLimit):
    ent_node = find_entity(ent)
    if ent_node:
        return ent_node
    if not find_entity(ent):
        label = ent.type
        d = deepcopy(ent.__dict__)
        d.pop("type")
        return create_node(graph, label, **d)

In [21]:
sub = get_node(rel.subject)
print(sub)

(_0:任务 {text: '\u5927\u5c3a\u5ea6\u751f\u6001\u7a7a\u95f4'})


In [14]:
obj = get_node(rel.object)
print(obj)

(_1:Object {text: '\u5317\u8fd0\u6cb3\u6e7f\u5730\u516c\u56ed'})


创建三元组

In [26]:
# 查询两个实体间的关系是否存在
relations = relation_matcher.match([sub, obj], r_type="包含").first()  # 可迭代对象

In [50]:
relation_matcher.match([sub, obj], r_type="包含").first()

包含(Node('任务', text='大尺度生态空间'), Node('Object', text='北运河湿地公园'))

In [27]:
rel_t = Relationship(sub, rel.relation, obj)
graph.create(rel_t)

查询刚才创建的关系是否存在

In [28]:
relations = relation_matcher.match([sub, obj])  # 可迭代对象
for relation in relations:
    print(relation)

(_0)-[:包含 {}]->(_1)


In [29]:
r = relation_matcher.match([sub, obj]).first()

In [30]:
r

包含(Node('任务', text='大尺度生态空间'), Node('Object', text='北运河湿地公园'))

In [31]:
r.__dict__

{'_Walkable__sequence': (Node('任务', text='大尺度生态空间'),
  包含(Node('任务', text='大尺度生态空间'), Node('Object', text='北运河湿地公园')),
  Node('Object', text='北运河湿地公园')),
 '_Subgraph__nodes': frozenset({Node('任务', text='大尺度生态空间'),
            Node('Object', text='北运河湿地公园')}),
 '_Subgraph__relationships': frozenset({包含(Node('任务', text='大尺度生态空间'), Node('Object', text='北运河湿地公园'))}),
 '__uuid__': '854f0a2a-f4f5-4355-be47-a1daba0e39cf',
 '_stale': set(),
 '_graph': Graph('bolt://localhost:7687'),
 'identity': 0}

字符串以什么开头 startswith()

In [44]:
attrs = [s for s in dir(r) if not s.startswith('_')]

In [47]:
for att in attrs:
    print(att, getattr(r, att))

clear <bound method Entity.clear of 包含(Node('任务', text='大尺度生态空间'), Node('Object', text='北运河湿地公园'))>
copy <built-in method copy of 包含 object at 0x00000245D35939A0>
end_node (_1:Object {text: '\u5317\u8fd0\u6cb3\u6e7f\u5730\u516c\u56ed'})
fromkeys <built-in method fromkeys of type object at 0x00000245CF141400>
get <built-in method get of 包含 object at 0x00000245D35939A0>
graph Graph('bolt://localhost:7687')
identity 0
items <built-in method items of 包含 object at 0x00000245D35939A0>
keys <built-in method keys of 包含 object at 0x00000245D35939A0>
labels <bound method Subgraph.labels of 包含(Node('任务', text='大尺度生态空间'), Node('Object', text='北运河湿地公园'))>
nodes (Node('任务', text='大尺度生态空间'), Node('Object', text='北运河湿地公园'))
pop <built-in method pop of 包含 object at 0x00000245D35939A0>
popitem <built-in method popitem of 包含 object at 0x00000245D35939A0>
ref <bound method Relationship.ref of <class 'py2neo.data.包含'>>
relationships (包含(Node('任务', text='大尺度生态空间'), Node('Object', text='北运河湿地公园')),)
setdefau

In [48]:
r.relationships

(包含(Node('任务', text='大尺度生态空间'), Node('Object', text='北运河湿地公园')),)

In [49]:
type(r.relationships)

tuple